<div>
<img src="https://www.ul.ie/themes/custom/ul/logo.jpg" />
</div>

# **MSc in Artificial Intelligence and Machine Learning**
## CS6482 - Deep Reinforcement Learning
### Assignment 2: Sem2 AY 23/24 - DQN Classic Control

Module Leader: J.J. Collins

Students:  
- Pratik Verma - 23007575
- Siddharth Prince - 23052058

---
## Code checks
| Sl. No. | Test case                             | Status |
|---------|---------------------------------------|--------|
| 1.      | [Imports and setup](#Imports-and-setup)  | <span style='color: orange'>In progress</span> |
| 2.      | [Capture and sampling of the data](#Capture-and-sampling-of-data) | <span style='color: #3af'>Awaiting</span> |
| 3.      | Network structure and hyperparameters | <span style='color: #3af'>Awaiting</span> |
| 4.      | Q-learning updates | <span style='color: #3af'>Awaiting</span> |
| 5.      | Metrics and evalutation | <span style='color: #3af'>Awaiting</span> |
| 6.      | Finetuning | <span style='color: #3af'>Awaiting</span> |
|   6.1      | Experimenting with Double DQN | <span style='color: #3af'>Awaiting</span> |
|   6.2      | Experimenting with Dueling DQN | <span style='color: #3af'>Awaiting</span> |

## References
- [DQN Tutorial | Pytorch.org](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)

CS6482 Deep Reinforcement Learning
Template for Assignment 2 DQN CartPole Classic Control

This sample code is based on one of the recommended texts. A small number of errors were deliberately injected into the code.

If using this code as a basis for Option 1:
•	Correct the bugs.
•	Comment the code extensively.
•	Replace CartPole with a different Classic Control problem form gym.
•	Generate more informative plots i.e. plot running average.
•	Consider generating an animation.
•	Play around with hyperparameters and parameters to achieve ‘satisfactory’ performance.

A template will NOT be provided for Assignment 3. Use this template and modify the following (1) shape of obs and actions (2) network structure, (3) storing and processing of experience, etc.

## Imports and setup

In [1]:
import sys
assert sys.version_info >= (3,10)

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:45:30_PST_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0


In [10]:
import torch

# Get appropriate device for training
device = ( "cuda"     # CUDA is a programming interface for certain Nvidia GPUs
           if torch.cuda.is_available()
           else "mps"  # Kind of like CUDA for a mac, I guess? I don't use macs
           if torch.backends.mps.is_available()
           else "cpu" )   # If you end up with CPU, go use Google Colab and get a GPU

print(f"{device} device available")

cuda device available


## Capture and sampling of data